# Import Modules

In [1]:
# iPython Settings
%load_ext autoreload
%autoreload 2

# Python Modules
import sys
sys.path.insert(0, "..")

import copy

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from ase.visualize import view

import plotly.plotly as py
import plotly.graph_objs as go

# My Modules ******************************************************************
from dft_job_automat.job_analysis import DFT_Jobs_Analysis
from dft_job_automat.job_types_classes.dft_methods import DFT_Methods
from orr_reaction.orr_methods import ORR_Free_E_Plot, calc_ads_e
from orr_reaction.orr_methods import lowest_e_path, df_calc_adsorption_e, plot_all_states
from energetics.dft_energy import Element_Refs

from colors.colors import color_list

# Local Imports
from an_data_processing import load_df

Divisor is Energy class instance!!!


In [2]:
h2o_fe_corr = 0.013
h2_fe_corr = -0.122

def calculate_gibbs_correction(row, h2o_corr, h2_corr):

    adsorbate = row["adsorbate"]
    ads_corr = row["gibbs_correction"]

#     print(adsorbate)

    corr_tot = "NaN"
    if adsorbate == "ooh":
        corr_tot = ads_corr - 2 * h2o_corr + (3. / 2.) * h2_corr 
    elif adsorbate == "o":
        corr_tot = ads_corr - 1 * h2o_corr + (2. / 2.) * h2_corr
    elif adsorbate == "oh":
        corr_tot = ads_corr - 1 * h2o_corr + (1. / 2.) * h2_corr
    elif adsorbate == "h2o":
        corr_tot = ads_corr - 1 * h2o_corr + (0. / 2.) * h2_corr

    return(corr_tot)

In [3]:


# tmp = - 2 * h2o_fe_corr + (3. / 2.) * h2_fe_corr
# print(2 * h2o_fe_corr)
# print((3. / 2.) * h2_fe_corr)
# print(tmp)

In [4]:
df = load_df(from_file=True)
df["new__"] = df.apply(calculate_gibbs_correction, args=(h2o_fe_corr, h2_fe_corr), axis=1)

col_list = [
    'new__',
    'ads_e',
    'adsorbate',
#     'atom_type_num_dict',
#     'atoms_object',
#     'charge_history',
    'elec_energy',
#     'final_site',
#     'full_path',
    'gibbs_correction',
    'graph_site',
#     'init_atoms',
#     'magmom_history',
#     'max_revision',
    'notes',
#     'path',
#     'pdos_data',
#     'revision_number',
    'site',
#     'spinpol',
    'system',
#     'tree_level_labels',
#     'variable_list',
    ]


df = df[col_list]

df_master = df.copy()

Attempting to load df from pickle


In [5]:
grouped = df.groupby("system")
master_data_list = []
for group_i in grouped:
    df = group_i[1]

    system_i = group_i[0]
    if system_i == "Fe_slab":
        color_i = color_list[0]
    elif system_i == "N_graph_Fe":
        color_i = color_list[1]
    elif system_i == "graphene":
        color_i = color_list[2]
    else:
        pass
    if "gibbs_correction" not in list(df):
        continue
    
    groupby = ["adsorbate"]
    data_master = {}
    
    data_list_j = []
    for group_i in df.groupby(groupby):
#         print(group_i[0])
        group_i = group_i[1][group_i[1]["gibbs_correction"] > 0.]
        gibbs_corr_i = group_i["gibbs_correction"].tolist()
        data = [
            go.Bar(
                x=group_i["adsorbate"], # assign x as the dataframe column 'x'
#                 y=group_i["gibbs_correction"],
                y=group_i["new__"],
                name=system_i,
                width = 0.25,
#                 showlegend=False,
                marker=dict(
                    color=color_i,
#                     line=dict(
#                         color='rgb(8,48,107)',
#                         width=1.5,
                    ),
          
            )]

        data_list_j = data_list_j + data
        
        # TODO Plot this along with the bar graph
        mean_fe_corr = group_i["gibbs_correction"].mean()
    master_data_list += data_list_j


In [6]:
group_i

,new__,ads_e,adsorbate,elec_energy,gibbs_correction,graph_site,notes,site,system
24,0.142416,5.305304,ooh,-1846.612104,0.351416,NaN,Desorbed,ontop,graphene
25,0.068616,5.305085,ooh,-1846.612138,0.277616,NaN,Desorbed,ontop,graphene
26,0.100600,5.250435,ooh,-1846.666973,0.309600,NaN,Desorbed,bridged,graphene
27,0.081957,5.314396,ooh,-1846.602827,0.290957,NaN,Desorbed,bridged,graphene
28,0.100807,5.238015,ooh,-1846.679393,0.309807,NaN,Desorbed,ring-center,graphene
29,0.096817,5.237423,ooh,-1846.679800,0.305817,NaN,Desorbed,ring-center,graphene


In [7]:
layout = dict(
    title = 'Free-energy Corrections',
   
    xaxis=dict(
        title="ORR Species",
        ),

    yaxis=dict(
        title="Free-energy Correction [eV]",
        ),
    )   

fig = dict(data=master_data_list, layout=layout)
py.iplot(fig, filename='01_fe_graph/pl_fe_corr_orr')

In [8]:
list(df)

['new__',
 'ads_e',
 'adsorbate',
 'elec_energy',
 'gibbs_correction',
 'graph_site',
 'notes',
 'site',
 'system']

In [9]:
df = df_master.copy()

master_data_list_x = []
master_data_list_y = []

master_data_list = []
for index, row in df.iterrows():
 
    master_data_list_x.append(row["adsorbate"])
    master_data_list_y.append(row["new__"])
    
#     name_i = row["adsorbate"] + " | " + row["system"] + " | " + row["site"]
    name_i = row["system"]
    
    data = [
        go.Bar(
            x=row["adsorbate"],
            y=row["new__"],
            name=name_i,
#             width = 0.1,
        )
    ]
    
    master_data_list += data
    

In [10]:
layout = dict(
    title = 'Free-energy Corrections',
    barmode = "group",
    xaxis=dict(
        title="ORR Species",
        ),

    yaxis=dict(
        title="Free-energy Correction [eV]",
        ),
    )   

fig = dict(data=master_data_list, layout=layout)
py.iplot(fig, filename='__temp__/pl_fe_corr_orr')

In [11]:
df = df_master.copy()

grouped = df.groupby("adsorbate")
master_data_list = []
for group_i in grouped:
    df_i = group_i[1]

In [12]:


data = [
    go.Bar(
        x=["A", "B"],
        y=[3.2, 2.2],
        name="temp",
    ),

    go.Bar(
        x=["B"],
        y=[1.8, 2.6],
        name="temp",
    ),

    go.Bar(
        x=["A", "B"],
        y=[4, 4.1],
        name="temp",
    ),

    go.Bar(
        x=["A", "B"],
        y=[1, 2],
        name="temp",
    ),

    
    go.Bar(
        x=["B"],
        y=[4],
        name="temp",
    ),

    
    go.Bar(
        x=["C"],
        y=[1],
        name="temp",
    ),

    go.Bar(
        x=["C"],
        y=[1],
        name="temp",
    ),

    go.Bar(
        x=["B", "C"],
        y=[3.1, 4.4],
        name="temp",
    ),


]

In [13]:
layout = dict(
    title = 'Free-energy Corrections',
    barmode = "group",
    xaxis=dict(
        title="ORR Species",
        ),

    yaxis=dict(
        title="Free-energy Correction [eV]",
        ),
    )   

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='__temp__/pl_fe_corr_orr')